In [1]:
import pandas as pd
import numpy as np

import category_encoders as ce

In [2]:
# Transform into ratio
filter_cols_pre = [
"renewvia_id", "avg_monthly_household_income",
"household_headcount", "female_schooling",
"male_schooling", "household_business_owners",
"electronics_count", "cellphones_count",
"light_hours_current", "kerosene_lamp_usage_count",
"cooking_energy_sources", "cooking_fuel_collection_time",
"cooking_energy_monthly_cost", "applicances_charging_monthly_cost",
"feel_safe_dark", "community_lights", "home_exterior_lights",
"feel_safe_if_exterior_lights", 
# "phone_charge_location",
"phone_charge_monthly_cost", "water_source",
"clean_drinking_water", "clean_drinking_water_source",
"water_collection_travel_distance", "water_collection_time",
"water_monthly_cost", "clinic_travel_distance",
"clinic_electricity_access", 
    # "clinic_open_hours",
"clinic_refrigeration_access",
]

#Importing the datasets
hs_pre_com = pd.read_csv("datasets_clean/household_pre_connection_commcare.csv", 
                         usecols=filter_cols_pre)
hs_pre_ms = pd.read_csv("datasets_clean/household_pre_connection_ms_form.csv", 
                        usecols=filter_cols_pre, encoding = "ISO-8859-1")
df = pd.concat([hs_pre_com, hs_pre_ms])
df.replace('nan', np.nan, inplace=True)
df.head()

,renewvia_id,avg_monthly_household_income,household_headcount,female_schooling,male_schooling,household_business_owners,electronics_count,cellphones_count,light_hours_current,kerosene_lamp_usage_count,...,phone_charge_monthly_cost,water_source,clean_drinking_water,clean_drinking_water_source,water_collection_travel_distance,water_collection_time,water_monthly_cost,clinic_travel_distance,clinic_electricity_access,clinic_refrigeration_access
0,501121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,570063,27000.0,3.0,NaN,1.0,adult_female,5.0,3.0,5.0,NaN,...,copy-2-of-nkes,clear_water_source_fresh_spring_lake_etc,yes,bottled_water,less_than_1_km,less_than_1_hour,nkes,less_than_1_km,no,no
2,570028,30000.0,9.0,2.0,4.0,adult_female,5.0,8.0,5.0,NaN,...,copy-3-of-nkes,at_home_tap,yes,boiled_water,less_than_1_km,NaN,i_dont_pay_its_free,less_than_1_km,no,no
3,NaN,30000.0,2.0,NaN,NaN,adult_female,4.0,2.0,5.0,NaN,...,nkes,clear_water_source_fresh_spring_lake_etc,yes,clean_community_source,less_than_1_km,less_than_1_hour,i_dont_pay_its_free,less_than_1_km,no,no
4,570097,38000.0,7.0,2.0,3.0,NaN,5.0,7.0,6.0,NaN,...,copy-3-of-nkes,at_home_tap,yes,bottled_water,km,less_than_1_hour,i_dont_pay_its_free,less_than_1_km,no,no


In [3]:
# No. of business owners
df['business_owners_count'] = df['household_business_owners'].map(lambda x: len(x.split(";")) 
                                                                  if type(x) == str else 0)
df['business_owners_female'] = df['household_business_owners'].map(lambda x: 1 
                                                                            if (type(x) == str) 
                                                                            and ('adult_female' in x) 
                                                                            else 0)

# standardize case
df = df.apply(lambda x: x.astype(str).str.lower())

cols_change = [
    'cooking_fuel_collection_time',
     'cooking_energy_monthly_cost',
     'applicances_charging_monthly_cost',
     'phone_charge_monthly_cost',
     'feel_safe_dark',
     'feel_safe_if_exterior_lights',
     'community_lights','water_source',
     'clean_drinking_water_source',
     'water_collection_travel_distance',
    'water_collection_time',
    'water_monthly_cost',
    'clinic_travel_distance']

def replace_str(string):
    replace_chars = {'_': ['- ',' (', ' ', '-'],
                    '': ['.)', ',', '/', 'u\n', "'", "."]
                    }

    for new, old in replace_chars.items():
        for c in old:
            string = string.replace(c, new)

    return string

for col in cols_change:
    df[col] = df[col].apply(lambda x: replace_str(x))
    


In [4]:
# replace values
replace_mapping = {
    'cooking_fuel_collection_time': [['copy_1_of_hours','hours'],
                                     ['less_than_1_hour',  '3_5_hours',]],
    
    'cooking_energy_monthly_cost' : [['0_1000_knes', 'copy_1_of_nkes', 
                                      '1000_1500_knes', 'copy_2_of_nkes', 
                                      '1500_2000_knes', 'copy_3_of_nkes',
                                      '2000_3000_knes','copy_4_of_nkes',
                                     'nkes', ], 
                                    ['0_1000_nkes', '0_1000_nkes', 
                                     '1000_1500_nkes', '1000_1500_nkes',
                                    '1500_2000_nkes', '1500_2000_nkes', 
                                    '2000_3000_nkes', '2000_3000_nkes',
                                    '3000_4000_nkes']],
    
    'applicances_charging_monthly_cost' : [['copy_1_of_nkes', 'copy_2_of_nkes', 
                                          'copy_3_of_nkes','copy_4_of_nkes',
                                           'nkes',], 
                                            ['0_150_nkes', '150_1000_nkes', 
                                            '1000_3000_nkes', '3000_4000_nkes',
                                            '4000_6000_nkes']],
    
    'phone_charge_monthly_cost': [['copy_1_of_nkes', 'copy_2_of_nkes', 'copy_3_of_nkes', 
                                   'nkes', 'nkes_and_above', '1000_n_and_above', 
                                  ], 
                                    ['0_100_nkes','100_500_nkes', '500_750_nkes',
                                     '750_1000_nkes',
                                     '1000_nkes_and_above', '1000_nkes_and_above']],
    
    'community_lights': [['street_lights', 'no_none', 'other'], 
                        ['yes', 'no', 'no']],
    
    'clean_drinking_water_source': ['clean_community_source_','clean_community_source',],
    
    'water_collection_travel_distance': [['copy_1_of_km','copy_2_of_km','km', ],
                                         ['less_than_1_km',  '1_2_km', '5_10_km']],
    
    'water_collection_time': [['copy_1_of_hours','hours'],
                             ['less_than_1_hour',  '3_4_hours',]],
    
    'water_monthly_cost': [['copy_1_of_nkes', 'copy_2_of_nkes', 
                            'nkes', 'nkes_and_above'], 
                          ['i_dont_pay_its_free', '0_500_nkes', 
                          '3000_5000_nkes', '5000_nkes_and_above',]],
    'clinic_travel_distance': ['less_than_1km', 'less_than_1_km']
    
}

for col, mapping in replace_mapping.items():
    df[col] = df[col].replace(mapping[0], mapping[1])

# list(replace_mapping.keys())
                           
# for col in list(df.columns)[5:]:
#     cats = df[col].unique()
#     print("\t ",col, list(cats))

df.head()

,renewvia_id,avg_monthly_household_income,household_headcount,female_schooling,male_schooling,household_business_owners,electronics_count,cellphones_count,light_hours_current,kerosene_lamp_usage_count,...,clean_drinking_water,clean_drinking_water_source,water_collection_travel_distance,water_collection_time,water_monthly_cost,clinic_travel_distance,clinic_electricity_access,clinic_refrigeration_access,business_owners_count,business_owners_female
0,501121,nan,nan,nan,nan,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,0,0
1,570063,27000.0,3.0,nan,1.0,adult_female,5.0,3.0,5.0,nan,...,yes,bottled_water,less_than_1_km,less_than_1_hour,3000_5000_nkes,less_than_1_km,no,no,1,1
2,570028,30000.0,9.0,2.0,4.0,adult_female,5.0,8.0,5.0,nan,...,yes,boiled_water,less_than_1_km,nan,i_dont_pay_its_free,less_than_1_km,no,no,1,1
3,nan,30000.0,2.0,nan,nan,adult_female,4.0,2.0,5.0,nan,...,yes,clean_community_source,less_than_1_km,less_than_1_hour,i_dont_pay_its_free,less_than_1_km,no,no,1,1
4,570097,38000.0,7.0,2.0,3.0,nan,5.0,7.0,6.0,nan,...,yes,bottled_water,5_10_km,less_than_1_hour,i_dont_pay_its_free,less_than_1_km,no,no,0,0


In [5]:
# Convert to numeric columns
cols_num = ['avg_monthly_household_income', 'household_headcount', 
            'female_schooling', 'male_schooling', 'electronics_count', 
           'cellphones_count', 'light_hours_current', 'kerosene_lamp_usage_count',
           'business_owners_count', 'business_owners_female']

for col in cols_num:
    # print(col)
    # df[col] = df[col].fillna(-1)
    df[col] = df[col].astype(float).astype('Int64')
    

# Encoding binary variables
cols_cat_bin = ['community_lights', 'home_exterior_lights', 
                'clinic_electricity_access', 
                'clinic_refrigeration_access',
                'business_owners_female']

#Binary Encoding
enc_bin = ce.BinaryEncoder(cols=cols_cat_bin, 
                            handle_unknown="return_nan",
                            handle_missing="return_nan", 
                            return_df=True)

df = enc_bin.fit_transform(df)

# cols_cat_nom = ['cooking_energy_sources', 'phone_charge_location']

In [7]:
cols_ord_map = {
'cooking_fuel_collection_time': ['less_than_1_hour', '1_2_hours', 
                                 '3_5_hours', 'greater_than_5_hours', ],
'cooking_energy_monthly_cost':  ['0_1000_nkes', '1000_1500_nkes', 
                                 '1500_2000_nkes','2000_3000_nkes', 
                                 '3000_4000_nkes'],
'applicances_charging_monthly_cost': ['0_150_nkes', '150_1000_nkes', 
                                     '1000_3000_nkes', '3000_4000_nkes', 
                                    '4000_6000_nkes'],
'feel_safe_dark': ['very_unsafe', 'somewhat_unsafe', 
                  'neither_safe_nor_unsafe', 
                   'somewhat_safe','very_safe',],
'feel_safe_if_exterior_lights': ['very_unsafe', 'somewhat_unsafe', 
                                  'neither_safe_nor_unsafe', 
                                 'somewhat_safe','very_safe',], 
'water_source': ['dirty_water_source_pond_contaminated_well_etc', 
                 'clear_water_source_fresh_spring_lake_etc', 
                 'community_well_or_pump',
                 'at_home_tap', ],    
'water_collection_travel_distance': ['less_than_1_km', '1_2_km', 
                                      '2_5_km', '5_10_km',
                                      'greater_than_10_km', ],
'water_collection_time':  ['less_than_1_hour', '1_2_hours', 
                           '2_3_hours', '3_4_hours', 
                           'greater_than_4_hours', ],
'water_monthly_cost':  ['i_dont_pay_its_free', '0_500_nkes',  
                        '500_3000_nkes', '3000_5000_nkes',
                        '5000_nkes_and_above',],
'clinic_travel_distance':  ['less_than_1_km', 'between_1_2_km', 
                            'between_2_3_km', 'between_3_5_km',
                           'greater_than_5_km', ]
#'phone_charge_monthly_cost'
}
cats_ord_map = list()
for col, values in cols_ord_map.items():
    val_ord = dict()
    rng = len(values)
    for idx, val in enumerate(values):
        val_ord[val] = idx+1
    cats_ord_map.append({"col":col, "mapping": val_ord})

    
#Ordinal Encoding
enc_ord = ce.OrdinalEncoder(mapping=cats_ord_map, 
                            handle_unknown="return_nan",
                            handle_missing="return_nan",
                            return_df=True)

df = enc_ord.fit_transform(df)
# print(df.info())
# df.head()
df.to_csv("datasets_clean/household_pre_survey_clean.csv")